In [5]:
import pandas as pd
import re # Thư viện để xử lý biểu thức chính quy (loại bỏ ký tự đặc biệt)
import string # Thư viện chứa các dấu câu

# Tải dữ liệu
df = pd.read_csv('../data/phishing_email.csv')

print("Tải dữ liệu thành công!")
df.info()

Tải dữ liệu thành công!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82486 entries, 0 to 82485
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text_combined  82486 non-null  object
 1   label          82486 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [6]:
# --- 1.1: Xử lý các giá trị bị thiếu (Missing Values) ---
# Kiểm tra xem có dòng nào bị thiếu dữ liệu không
print(f"Số dòng bị thiếu dữ liệu trước khi xử lý: {df.isnull().sum().sum()}")
df.dropna(inplace=True) # Xóa tất cả các dòng có giá trị thiếu
print("Đã xóa các dòng bị thiếu dữ liệu (nếu có).")


# --- 1.2: Loại bỏ dữ liệu trùng lặp (Duplicates) ---
# Kiểm tra xem có bao nhiêu dòng bị trùng lặp hoàn toàn
print(f"\nSố dòng bị trùng lặp trước khi xử lý: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True) # Xóa các dòng bị trùng lặp
print("Đã xóa các dòng bị trùng lặp.")
print(f"Số dòng còn lại sau khi làm sạch: {len(df)}")


# --- 1.3: Tiền xử lý văn bản (Text Preprocessing) ---
# Chúng ta sẽ viết một hàm để làm sạch văn bản
def preprocess_text(text):
    # Chuyển thành chữ thường
    text = text.lower()
    # Loại bỏ các ký tự đặc biệt, dấu câu
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    # Loại bỏ các số (thường không mang nhiều ý nghĩa trong phishing)
    text = re.sub(r'\d+', '', text)
    # Loại bỏ khoảng trắng thừa
    text = ' '.join(text.split())
    return text

print("\nBắt đầu tiền xử lý văn bản...")
# Áp dụng hàm này cho toàn bộ cột 'text_combined'
df['clean_text'] = df['text_combined'].apply(preprocess_text)
print("Tiền xử lý văn bản hoàn tất!")

# In ra để so sánh văn bản gốc và văn bản đã làm sạch
print("\n--- VÍ DỤ TIỀN XỬ LÝ ---")
print("Văn bản gốc:\n", df['text_combined'].iloc[0][:300])
print("\nVăn bản đã làm sạch:\n", df['clean_text'].iloc[0][:300])

Số dòng bị thiếu dữ liệu trước khi xử lý: 0
Đã xóa các dòng bị thiếu dữ liệu (nếu có).

Số dòng bị trùng lặp trước khi xử lý: 408
Đã xóa các dòng bị trùng lặp.
Số dòng còn lại sau khi làm sạch: 82078

Bắt đầu tiền xử lý văn bản...
Tiền xử lý văn bản hoàn tất!

--- VÍ DỤ TIỀN XỬ LÝ ---
Văn bản gốc:
 hpl nom may 25 2001 see attached file hplno 525 xls hplno 525 xls

Văn bản đã làm sạch:
 hpl nom may see attached file hplno xls hplno xls


In [7]:
from sklearn.model_selection import train_test_split

# Bây giờ chúng ta sẽ dùng cột 'clean_text' đã được làm sạch
X = df['clean_text']
y = df['label']

# Chia dữ liệu thành 2 bộ: 80% để huấn luyện, 20% để kiểm thử
# test_size=0.2 có nghĩa là 20% cho tập test
# random_state=42 để đảm bảo mỗi lần chạy, kết quả chia đều giống nhau, giúp dễ dàng tái tạo thí nghiệm
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Đã chia dữ liệu thành các bộ train và test:")
print(f"Số lượng mẫu trong tập huấn luyện (train): {len(X_train)}")
print(f"Số lượng mẫu trong tập kiểm thử (test): {len(X_test)}")

Đã chia dữ liệu thành các bộ train và test:
Số lượng mẫu trong tập huấn luyện (train): 65662
Số lượng mẫu trong tập kiểm thử (test): 16416


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# --- 3. "Số hóa" văn bản (Vectorization) ---
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("\nĐã hoàn tất việc số hóa văn bản!")

# --- 4. Huấn luyện và Đánh giá ---
model = LogisticRegression(max_iter=1000)

print("Bắt đầu huấn luyện...")
model.fit(X_train_tfidf, y_train)
print("Huấn luyện hoàn tất!")

predictions = model.predict(X_test_tfidf)

print("\n--- KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG ---")
print(classification_report(y_test, predictions, target_names=['Safe Email (0)', 'Phishing Email (1)']))


Đã hoàn tất việc số hóa văn bản!
Bắt đầu huấn luyện...
Huấn luyện hoàn tất!

--- KẾT QUẢ ĐÁNH GIÁ CUỐI CÙNG ---
                    precision    recall  f1-score   support

    Safe Email (0)       0.99      0.98      0.99      7910
Phishing Email (1)       0.99      0.99      0.99      8506

          accuracy                           0.99     16416
         macro avg       0.99      0.99      0.99     16416
      weighted avg       0.99      0.99      0.99     16416

